In [2]:
# Import the necessary packages
import pandas as pd
import numpy as np
import re 
import geopandas as gpd
import pandas as pd
import json

### Load the Budget Data

In [5]:
filepath= 'data/demographics/budgets.csv'
budgets_df = pd.read_csv(filepath)
budgets_df.head()

,year,city_name,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,igr_state_city,...,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
0,1977,AK: Anchorage,22002001.0,174500,4.044885,5342.24,4956.92,2148.77,279.32,1869.46,...,178.51,787.93,691.32,NaN,NaN,NaN,4.0,36855.0,NaN,1.0
1,1978,AK: Anchorage,22002001.0,177000,3.759509,5948.99,5490.05,2468.11,403.24,2064.86,...,187.53,1395.82,1158.01,NaN,NaN,NaN,4.0,36804.0,NaN,1.0
2,1979,AK: Anchorage,22002001.0,179600,3.376308,6158.68,5746.64,2573.34,496.97,2076.37,...,249.03,1812.04,1314.62,NaN,NaN,NaN,4.0,36757.0,NaN,1.0
3,1980,AK: Anchorage,22002001.0,178800,2.974757,5654.93,5210.77,2313.62,371.46,1942.16,...,986.73,1330.72,1572.66,NaN,NaN,NaN,4.0,36008.0,NaN,1.0
4,1981,AK: Anchorage,22002001.0,174431,2.696590,6192.83,5736.81,2771.43,338.76,2432.67,...,977.32,1549.51,1669.55,NaN,NaN,NaN,4.0,34557.0,NaN,1.0


### About the Data

This data came from the the Lincoln Institute of Land Policy whose main mission is to improve quality of life through the effective use, taxation, and stewardship of land. This particular dataset is known as the [Fiscally Standardized Cities Database](https://www.lincolninst.edu/research-data/data-toolkits/fiscally-standardized-cities). It contains data for over 200 of the largest cities. The FiSC estimates are critical for making meaningful fiscal comparisons at the city level, because the delivery of public services is organized in very different ways in different cities. 

While some city governments provide a full array of public services for their residents and businesses, others share the responsibility with a variety of overlying independent governments. Fiscal comparisons across central city municipal governments alone can thus be highly misleading.

A more detailed explanation can be found at the above link. However in a nutshell the FISC database accounts for differences across cities in government structure, making it possible to compare many aspects of local government finances for the nation’s largest cities.

It contains data from 1977 to 2017. The FiSC database was constructed using information from the individual unit of government files produced by the Governments Division of the U.S. Census Bureau.

In the dataset, all values are expressed in real per capita dollars ($2017). However, it is possible to convert to nominal dollars (Not adjusted for inflation) and/or total dollars (Not adjusted for population changes) by using variables in the dataset for City Population (Column D) and/or the Consumer Price Index (Column E).

```
Real Total Dollars = (Fiscal Variable) x (City Population)
Nominal Per Capita Dollars = (Fiscal Variable) / (CPI)
Nominal Total Dollars = [(Fiscal Variable) x (City Population)] / (CPI)
```

### Analyze Columns

List of Comparison Variables: 
- rev_total: Total Revenue 
- rev_general: General Revenue
- intergovt_rev: Intergovernmental Revenue
- igr_federal: Revenue from Federal Government 
- igr_state: Revenue from 
- own_source_rev: Own Source of Revenue
- taxes: **Total taxes**
    - tax_property
    - tax_sales_grossrec
    - tax_sales_general
    - tax_sales_selectiv
    - tax_sales_motofuel
    - tax_sales_alcohol
    - tax_sales_tobacco
    - tax_sales_utility
    - tax_sales_other
    - tax_income_indiv
    - tax_income_corp
    - tax_motor_vehicle
    - tax_other
    
 - Charges_misc_rev: **Charges and Miscellaneous Revenue**
    - charges: Total Charges
    - chg_educ
    - chg_higher_ed
    - chg_school_lunch
    - chg_hospitals
    - chg_highways
    - chg_air_transit
    - chg_parking
    - chg_seaport
    - chg_nat_resources
    - chg_parks_rec
    - chg_hous_commdev: Housing and Community Development
    - chg_sewers
    - chg_waste_mgmt
    - chg_other
- misc_general_rev: **Miscellaneous General Revenue**
    - misc_interest
    - misc_fines_fees
    - misc_spec_assess
    - misc_property_sale
    - misc_other
- rev_utility: **Revenues from Utilities**
    - rev_utility_water
    - rev_utility_elec
    - rev_utility_gas
    - rev_utility_transit
    - rev_liquor_store
    - rev_empl_retire_tr

- spending_direct: **Direct Expenditures**
    - spending_general
    - education_services
    - education
    - educ_higher
    - educ_elem_sec
    - libraries
    - social_services
    - public_welfare
    - welfare_cash
    - welfare_vendors
    - welfare_other
    - hospitals
    - health
    - transportation
    - highways
    - air_transit
    - parking
    - seaport
    - public_safety
    - police
    - fire
    - correction
    - inspection
    - envir_housing
    - natural_resources
    - parks_recreation
    - housing_commdevt
    - sewerage
    - waste_mgmt
    - administration
    - admin_financial
    - admin_judicial
    - public_buildings
    - admin_other
    - interest_general
    - misc
    - commercial_activ
    - other_gen_spending
    - spending_utility
    - utility_water
    - utility_elec
    - utility_gas
    - utility_transit
    - spend_liquor_store
    - sp_empl_retire_tr

- spending_total: **Total Expenditures**
    - spend_intergovt
    - spending_direct
    - spend_current_ops
    - spend_capital
    - capital_construct
    - capital_other_dir
    - spend_assistance
    - spend_debtinterest
    - spend_insurance
    - salaries_wages

- spend_capital: **Capital Outlay**: Capital outlay is defined as money that's spent to maintain, upgrade, acquire, or repair capital assets.
    - capital_general
    - capital_education
    - capital_higher_ed
    - capital_elem_ed
    - capital_hospitals
    - capital_highways
    - capital_correction
    - capital_nat_res
    - capital_parks_rec
    - capital_sewers
    - capital_waste_mgmt
    - capital_other_gen
    - capital_utility


- debt_outstanding: **Debt and Assets**
    - debt_short_term
    - debt_long_term
    - debt_private_purp
    - debt_issued
    - debt_retired
    - cash_holdings
    - cash_empl_retire
    - cash_other
    - cash_other_offsets
    - cash_other_bonds
    - cash_other_other